<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/Seq2SeqTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [5]:
all_data = []
resp = urlopen("https://download.pytorch.org/tutorial/data.zip")
zipfile = ZipFile(BytesIO(resp.read()))
for line in zipfile.open('data/eng-fra.txt').readlines():
  line = line.decode('utf-8').lower()
  line = re.sub(r"[-.!?]+", r" ", line) # remove punctuation.
  source_lang, target_lang = line.split('\t')[0:2]
  all_data.append( (source_lang.strip(), target_lang.strip()) )

In [7]:
short_subset = []
MAX_LEN = 6
for (s, t) in all_data:
  if max( len(s.split(" ")), len(t.split(" ")) ) <= MAX_LEN:
    short_subset.append((s, t))

print("Using ", len(short_subset), "/", len(all_data))

Using  66251 / 135842


In [8]:
SOS_token = '<SOS>'  # Start of Sentence
EOS_token = "<EOS>"  # End of Sentence
PAD_token = "_PADDING_"  # Padding Token

word2index = {PAD_token:0, SOS_token:1, EOS_token:2}

for s, t in short_subset:
  for sentence in (s, t):
    for word in sentence.split(" "):
      if word not in word2index:
        word2index[word] = len(word2index)
print('size of vocab ', len(word2index))

indextoword = {}
for word, index in word2index.items():
  indextoword[index] = word



size of vocab  24577
